<a href="https://colab.research.google.com/github/noorinbinary/blank-app/blob/main/OrionDummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community requests
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA


In [3]:
import requests
import yaml
!pip install groq


from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



In [4]:
import os
GROQ_API_KEY="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"
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")



In [6]:
!pip install python-docx unstructured[docs] docx2txt


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.4 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=dc100b766738b6742db2cecdfad48e2ddb35baf853faf0dae250851d225cc8ec
  Stored in directory: /root/.cache/pip/wheels/0f/0e/7a/3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


In [7]:
from langchain.document_loaders import DirectoryLoader, Docx2txtLoader
loader=DirectoryLoader("sample_data/",glob="*.docx",loader_cls=Docx2txtLoader)
documents =loader.load()

In [8]:
from langchain.text_splitter import CharacterTextSplitter


In [9]:
text_splitter = CharacterTextSplitter (
    chunk_size=2000,
    chunk_overlap=500

)
text_chunks=text_splitter.split_documents(documents)

In [10]:
embedding=HuggingFaceEmbeddings()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
persist_directory="doc_db"

In [12]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [13]:
retriever=vectorstore.as_retriever()

In [14]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=500
)

In [15]:
from langchain.prompts import PromptTemplate
custom_prompt=PromptTemplate(
    input_variables=["context","question"],
    template="Use only the following Document excerpts :\n{context}\n\n"
    "Question:{question}\n"
    "Answer Strictly based on the excerpts.If not found Say not available in the Modules "
)

qa_chain=RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=retriever,
      return_source_documents =True,
      chain_type_kwargs={"prompt":custom_prompt}
  )

In [20]:
query =" can you give me an example of i2c"
response=qa_chain.invoke({"query":query})


In [21]:
print(response["result"])

Here's an example of I2C communication based on the provided excerpts:

**I2C Wiring Diagram**

MPU6050 Pin | Arduino Pin | Function
-----------|-------------|---------
VCC        | 5V          | Powers the MPU6050
GND        | GND         | Ground connection
SCL        | A5 (SCL)    | Clock line for I2C
SDA        | A4 (SDA)    | Data line for I2C
INT        | (Optional)  | Interrupt signal

**How I2C Works**

SDA (Serial Data): Transfers data between devices.
SCL (Serial Clock): Synchronizes data transmission.
Address: The default I2C address for most SSD1306 displays is 0x3C.

**Example Code**

```cpp
#include <Wire.h>

void setup() {
  Wire.begin(); // Initialize I2C communication
}

void loop() {
  Wire.beginTransmission(0x3C); // Start I2C transmission to address 0x3C
  Wire.write("Hello, I2C!"); // Send data over I2C
  Wire.endTransmission(); // End I2C transmission
  delay(1000); // Wait 1 second
}
```

Note: This is a simplified example and may not work as-is in your project. 